In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
%pwd

'/Users/terryjiang'

In [6]:
bank_bal=pd.read_csv('data.csv.zip')
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


# Data Sampling

In [8]:
bank_bal=bank_bal.groupby('Bankrupt?',group_keys=False).apply(lambda x: x.sample(1000,replace=True,random_state=0))
bank_bal['Bankrupt?'].value_counts()

0    1000
1    1000
Name: Bankrupt?, dtype: int64

In [9]:
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2893,0,0.616195,0.655037,0.642915,0.616325,0.616325,0.999192,0.797712,0.809545,0.303622,...,0.861914,0.002909,0.623981,0.616320,0.844897,0.278677,0.026794,0.565174,1,0.034458
2765,0,0.582850,0.621784,0.622196,0.642586,0.642586,0.999160,0.797665,0.809535,0.303607,...,0.844453,0.000395,0.623838,0.642585,0.842776,0.276194,0.026802,0.565209,1,0.074229
1731,0,0.500463,0.545464,0.547620,0.605198,0.605198,0.999033,0.797413,0.809341,0.303432,...,0.799789,0.001395,0.408682,0.605198,0.840362,0.280128,0.028611,0.568346,1,0.028003
3434,0,0.475455,0.545464,0.532684,0.598661,0.598675,0.998970,0.797396,0.809328,0.303534,...,0.802893,0.000755,0.623876,0.598659,0.840796,0.283750,0.027123,0.566364,1,0.020906
5125,0,0.518403,0.578772,0.562129,0.599158,0.599165,0.999015,0.797446,0.809354,0.303527,...,0.819451,0.005034,0.624249,0.599160,0.842552,0.283468,0.026844,0.565399,1,0.021249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,1,0.412616,0.422209,0.453557,0.598358,0.598358,0.998916,0.796831,0.808784,0.302660,...,0.678242,0.008544,0.623109,0.598354,0.829094,0.281469,0.026637,0.564297,1,0.024493
4264,1,0.407400,0.442652,0.438086,0.609349,0.609320,0.998786,0.797106,0.808996,0.303413,...,0.727838,0.000782,0.623797,0.609344,0.831255,0.286656,0.026645,0.564353,1,0.018316
1752,1,0.448447,0.463694,0.486000,0.594027,0.594027,0.998887,0.797178,0.809074,0.303328,...,0.748700,0.001631,0.624708,0.594027,0.834098,0.286193,0.026639,0.564309,1,0.018641
2704,1,0.495539,0.545901,0.547674,0.619770,0.619770,0.999041,0.797473,0.809367,0.303521,...,0.800066,0.002033,0.625247,0.619771,0.840303,0.277260,0.027737,0.567538,1,0.047776


# Logistic Regression¶

In [10]:
bank_bal[bank_bal['Bankrupt?']==1].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,1.0,0.42,0.45,0.46,0.60,0.60,1.00,0.80,0.81,0.30,...,0.73,1.276100e+08,0.62,0.60,0.82,0.29,0.03,0.56,1.0,0.02
std,0.0,0.08,0.11,0.09,0.01,0.01,0.00,0.00,0.01,0.01,...,0.09,1.012655e+09,0.01,0.01,0.08,0.07,0.01,0.01,0.0,0.04
min,1.0,0.02,0.00,0.03,0.53,0.53,0.99,0.76,0.76,0.24,...,0.41,0.000000e+00,0.59,0.53,0.00,0.00,0.02,0.55,1.0,0.00
25%,1.0,0.39,0.43,0.43,0.59,0.59,1.00,0.80,0.81,0.30,...,0.72,0.000000e+00,0.62,0.59,0.83,0.28,0.03,0.56,1.0,0.02
50%,1.0,0.44,0.49,0.49,0.60,0.60,1.00,0.80,0.81,0.30,...,0.76,0.000000e+00,0.62,0.60,0.84,0.29,0.03,0.56,1.0,0.02
75%,1.0,0.47,0.53,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.79,1.000000e-02,0.62,0.60,0.84,0.29,0.03,0.57,1.0,0.02
max,1.0,0.58,0.58,0.62,0.67,0.67,1.00,0.80,0.81,0.31,...,0.82,9.170000e+09,0.68,0.67,1.00,1.00,0.26,0.62,1.0,0.80


In [11]:
bank_bal[bank_bal['Bankrupt?']==0].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,1.651000e+07,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
std,0.0,0.06,0.06,0.06,0.01,0.01,0.00,0.00,0.00,0.00,...,0.03,3.742320e+08,0.01,0.01,0.00,0.01,0.02,0.02,0.0,0.05
min,0.0,0.23,0.16,0.21,0.45,0.45,0.97,0.75,0.77,0.28,...,0.53,0.000000e+00,0.41,0.45,0.78,0.27,0.00,0.17,1.0,0.01
25%,0.0,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.000000e+00,0.62,0.60,0.84,0.28,0.03,0.57,1.0,0.02
50%,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.000000e+00,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.03
75%,0.0,0.54,0.59,0.59,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,1.000000e-02,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
max,0.0,1.00,0.95,0.96,0.66,0.66,1.00,0.80,0.81,0.31,...,1.00,9.650000e+09,0.73,0.66,0.85,0.40,0.54,1.00,1.0,0.94


## controlling the complexity of log regression, using parameter 'c'

In [12]:
X_bank, y_bank=bank_bal.iloc[:,1:], bank_bal['Bankrupt?'] 
display(X_bank.head(2))
display(y_bank.head(2))

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2893,0.616195,0.655037,0.642915,0.616325,0.616325,0.999192,0.797712,0.809545,0.303622,0.781821,...,0.861914,0.002909,0.623981,0.616320,0.844897,0.278677,0.026794,0.565174,1,0.034458
2765,0.582850,0.621784,0.622196,0.642586,0.642586,0.999160,0.797665,0.809535,0.303607,0.781791,...,0.844453,0.000395,0.623838,0.642585,0.842776,0.276194,0.026802,0.565209,1,0.074229


2893    0
2765    0
Name: Bankrupt?, dtype: int64

## Feature Selection

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_bank,y_bank)

DecisionTreeClassifier(random_state=0)

In [15]:
sum(dt.feature_importances_>.01)

17

In [16]:
X_bank_sel=X_bank[X_bank.columns[dt.feature_importances_>.01]]
X_bank_sel.head()

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
2893,0.616325,0.999192,0.781821,1.414234e-04,0.471874,0.000511,0.630626,0.369638,0.041452,0.002231,0.276975,0.000000,0.953235,1.100000e+09,0.702181,0.002909,0.034458
2765,0.642586,0.999160,0.781791,5.949498e-04,0.474274,0.000504,0.630653,0.369884,0.027742,0.001130,0.277239,0.026619,0.947642,2.700000e+09,0.956182,0.000395,0.074229
1731,0.605198,0.999033,0.781601,1.217652e-04,0.461560,0.000529,0.633266,0.377127,0.024355,0.030132,0.277396,0.005834,0.932778,6.130000e+09,0.615365,0.001395,0.028003
3434,0.598675,0.998970,0.781587,8.040000e+09,0.461483,0.000455,0.631264,0.378270,0.060484,0.007349,0.277195,0.001691,0.933728,2.910000e+09,0.990300,0.000755,0.020906
5125,0.599165,0.999015,0.781616,4.290000e+09,0.471963,0.000484,0.630827,0.378462,0.089355,0.003401,0.277167,0.006017,0.938630,9.300000e+09,0.729870,0.005034,0.021249


## transform the features

In [17]:
scaler1=MinMaxScaler()
X_bank_trns=scaler1.fit_transform(X_bank_sel)
X_bank_trns

array([[7.73312812e-01, 9.82548201e-01, 9.16489802e-01, ...,
        6.86644392e-01, 2.90976116e-13, 3.25008656e-02],
       [8.95238733e-01, 9.81323358e-01, 9.15889703e-01, ...,
        9.53895586e-01, 3.04361447e-14, 7.48655416e-02],
       [7.21651554e-01, 9.76494484e-01, 9.12167984e-01, ...,
        5.95299014e-01, 1.34017374e-13, 2.56255361e-02],
       ...,
       [6.69789541e-01, 9.70907793e-01, 9.06535856e-01, ...,
        6.84193412e-01, 1.58532414e-13, 1.56528592e-02],
       [7.89306387e-01, 9.76785274e-01, 9.12861677e-01, ...,
        7.69072401e-01, 2.00108797e-13, 4.66873896e-02],
       [7.04286144e-01, 9.74058017e-01, 9.00086161e-01, ...,
        6.59380527e-01, 9.44858467e-13, 1.33390400e-02]])

## validation curve functions

In [18]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_trns, y_bank, test_size=0.2, random_state=0)

In [19]:
train_scores_log_reg, test_scores_log_reg=validation_curve(LogisticRegression(penalty='l1', solver='saga', max_iter=100000), X_train, y_train, param_name='C',
                                           param_range=[0.01,0.05,1,5,10,20], cv=4)

In [20]:
train_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.7585, 0.8235, 0.85  , 0.8529, 0.8542])

In [21]:
test_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.7556, 0.8181, 0.8444, 0.845 , 0.8462])

In [22]:
log_reg=LogisticRegression(max_iter=1000, C=20)
log_reg.fit(X_train, y_train)

LogisticRegression(C=20, max_iter=1000)

In [23]:
print('log reg acc on train: {:.2%}'.format(log_reg.score(X_train, y_train)))
print('log reg acc on test: {:.2%}'.format(log_reg.score(X_test, y_test)))

log reg acc on train: 83.50%
log reg acc on test: 85.00%


In [24]:
log_reg.coef_

array([[-6.91663087e+00,  8.79304840e+00, -5.27932152e+00,
        -1.28199976e-02, -1.02619857e+01,  6.22985190e-01,
         2.99142714e+00,  5.95006278e+00, -1.22106913e+00,
         2.74156076e+00, -7.11789114e+00,  4.07425044e-01,
        -1.51317366e+01, -1.65845483e-01, -5.96896452e-02,
        -1.60568421e+00, -1.76186155e+01]])

In [25]:
p1=[0.2,0.4,0.5,0.1,0.4,0.3,0.2,0.5,0.2,0.5,0.5,0.2,0.2,0.4,0.5,0.2,0.1]
log_reg.predict([p1])

array([1])

In [26]:
import numpy as np
np.set_printoptions(suppress=True)

In [27]:
log_reg.predict_proba([p1]).round(4)

array([[0.0011, 0.9989]])

In [28]:
log_reg.classes_

array([0, 1])

In [29]:
log_reg.coef_.round(4)

array([[ -6.9166,   8.793 ,  -5.2793,  -0.0128, -10.262 ,   0.623 ,
          2.9914,   5.9501,  -1.2211,   2.7416,  -7.1179,   0.4074,
        -15.1317,  -0.1658,  -0.0597,  -1.6057, -17.6186]])

In [30]:
log_reg.intercept_.round(4)

array([15.2447])

# SVM 

In [31]:
X_bank_trns

array([[0.77331281, 0.9825482 , 0.9164898 , ..., 0.68664439, 0.        ,
        0.03250087],
       [0.89523873, 0.98132336, 0.9158897 , ..., 0.95389559, 0.        ,
        0.07486554],
       [0.72165155, 0.97649448, 0.91216798, ..., 0.59529901, 0.        ,
        0.02562554],
       ...,
       [0.66978954, 0.97090779, 0.90653586, ..., 0.68419341, 0.        ,
        0.01565286],
       [0.78930639, 0.97678527, 0.91286168, ..., 0.7690724 , 0.        ,
        0.04668739],
       [0.70428614, 0.97405802, 0.90008616, ..., 0.65938053, 0.        ,
        0.01333904]])

In [32]:
svm1=LinearSVC(random_state=0)
svm1.fit(X_train,y_train)

LinearSVC(random_state=0)

In [33]:
svm1.coef_

array([[-2.3922309 ,  3.02109436, -1.27422496, -0.00527948, -2.20330322,
        -0.        ,  1.20639627,  3.02475093, -0.16313754,  0.89815626,
        -1.72630618,  0.0456347 , -5.17381964, -0.04322548, -0.00509823,
        -0.56900942, -3.13605435]])

In [34]:
svm1.intercept_

array([3.50851562])

In [35]:
print('svm acc on train: {:.2%}'.format(svm1.score(X_train, y_train)))
print('svm acc on test: {:.2%}'.format(svm1.score(X_test, y_test)))

svm acc on train: 81.94%
svm acc on test: 84.25%


In [36]:
b=[[0.616,0.999,0.781,0.00001,0.4719,0.0005,0.6306,0.3696,0.0415,0.0022,0.2770,0.9532,0,1.1,0.7021,0.0029,0.0345]]

In [37]:
b_trns=scaler1.transform(b)
b_trns

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[ 0.77180473,  0.97523232,  0.9003833 , -0.        ,  0.46878615,
         0.        ,  0.28705302,  0.3696    ,  0.08202934,  0.        ,
         0.14518228,  0.        , -2.69984748,  0.        ,  0.68655891,
         0.        ,  0.0325461 ]])

In [38]:
svm1.predict(b_trns)

array([1])

In [39]:
svm1.intercept_.round(2)

array([3.51])

In [40]:
svm1.coef_.round(2)

array([[-2.39,  3.02, -1.27, -0.01, -2.2 , -0.  ,  1.21,  3.02, -0.16,
         0.9 , -1.73,  0.05, -5.17, -0.04, -0.01, -0.57, -3.14]])

## controlling the complexity of svm by change parameter 'C'

In [41]:
LinearSVC()

LinearSVC()

In [42]:
C_range=[.1,1,5,10,50,100]

In [43]:
train_scores_SVM, test_scores_SVM=validation_curve(LinearSVC(random_state=0, max_iter=100000), X_bank_trns, y_bank, param_name='C',
                                           param_range=[.1,1,5,10,50,100], cv=4)

In [44]:
train_scores_SVM.round(4)

array([[0.7927, 0.818 , 0.786 , 0.79  ],
       [0.8233, 0.8247, 0.8193, 0.8167],
       [0.826 , 0.8273, 0.8293, 0.822 ],
       [0.83  , 0.826 , 0.8267, 0.8233],
       [0.8307, 0.8287, 0.8247, 0.8247],
       [0.8293, 0.8287, 0.8227, 0.8227]])

In [45]:
print('ave cross in 4 folds for different C values:')
print('training:', train_scores_SVM.mean(axis=1).round(4))
print('test', test_scores_SVM.mean(axis=1).round(4))

ave cross in 4 folds for different C values:
training: [0.7967 0.821  0.8262 0.8265 0.8272 0.8258]
test [0.797  0.8205 0.8225 0.8215 0.824  0.823 ]


In [46]:
train_scores_SVM.mean()

0.8205555555555555

# Decision tree

In [47]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, test_size=0.2, random_state=0)

In [48]:
X_train

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
1270,0.607698,0.999075,0.781581,1.204932e-04,0.465501,0.000447,0.634492,0.392679,0.037097,0.035477,0.276596,0.004880,0.917185,4.260000e+09,0.727656,0.006280,0.016797
6201,0.591022,0.998626,0.781222,3.020425e-04,0.478040,0.000429,0.629540,0.374118,0.012903,0.035610,0.276956,0.000185,0.943350,2.785699e-04,0.542416,0.000875,0.043125
1639,0.590856,0.998732,0.781208,1.927660e-04,0.458967,0.000388,0.629974,0.379328,0.024355,0.013620,0.276177,0.002407,0.917745,2.610000e+09,0.770937,0.012318,0.022717
3872,0.602524,0.999032,0.781622,7.990000e+08,0.484796,0.000414,0.633260,0.379072,0.023065,0.020758,0.277173,0.006388,0.943059,2.593929e-04,0.305700,0.009755,0.025943
5443,0.603511,0.999023,0.781639,1.045354e-04,0.469698,0.000438,0.630630,0.369637,0.026452,0.003649,0.277012,0.001440,0.938743,2.590000e+09,0.806450,0.001010,0.046239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4916,0.601183,0.999023,0.781632,6.760000e+09,0.460872,0.000424,0.630873,0.374808,0.036129,0.006172,0.277277,0.003578,0.945173,1.620000e+09,0.737087,0.004855,0.024575
6411,0.600989,0.998889,0.781362,1.984322e-04,0.458103,0.000367,0.630012,0.391831,0.049032,0.006413,0.278349,0.004280,0.919304,1.160000e+09,0.833007,0.004843,0.015417
4491,0.616930,0.999190,0.781618,1.525245e-04,0.463412,0.000484,0.633105,0.372200,0.012258,0.042561,0.277020,0.000580,0.932301,2.040000e+08,0.867257,0.021720,0.058527
3316,0.625168,0.999191,0.781735,2.840032e-04,0.483070,0.000465,0.630703,0.369637,0.018710,0.008820,0.277028,0.009467,0.950731,2.451023e-04,0.923759,0.020385,0.133604


In [49]:
train_scores_decision_tree, test_scores_decision_tree=validation_curve(DecisionTreeClassifier(random_state=0), X_train, y_train, param_name='max_leaf_nodes', 
                                          param_range=[2,6,8,10,15,25,50], cv=5)

In [50]:
dt0=DecisionTreeClassifier(random_state=0, max_leaf_nodes=10)
dt0.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [51]:
print('dt0 acc on train: {:.2%}'.format(dt0.score(X_train, y_train)))
print('dt0 acc on test: {:.2%}'.format(dt0.score(X_test, y_test)))

dt0 acc on train: 90.19%
dt0 acc on test: 89.50%


In [52]:
p1=[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]
dt0.predict([p1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])

## visulalzing our decision tree

In [53]:
export_graphviz(dt0, out_file='dt0_vis.dot', feature_names=X_bank_sel.columns, class_names=dt0.classes_.astype(str), impurity=False, filled=True)

In [54]:
dt0.classes_

array([0, 1])

## Feature Importance 

In [55]:
dt0.feature_importances_.round(3)

array([0.027, 0.043, 0.567, 0.   , 0.   , 0.049, 0.124, 0.   , 0.024,
       0.   , 0.   , 0.027, 0.   , 0.027, 0.   , 0.   , 0.111])

In [56]:
X_bank_sel.columns

Index([' Realized Sales Gross Margin', ' Operating Profit Rate',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Cash flow rate', ' Net Value Growth Rate', ' Interest Expense Ratio',
       ' Borrowing dependency', ' Net Worth Turnover Rate (times)',
       ' Allocation rate per person', ' Inventory/Working Capital',
       ' Inventory/Current Liability', ' Retained Earnings to Total Assets',
       ' Cash Turnover Rate', ' Current Liability to Liability',
       ' Total assets to GNP price', ' Equity to Liability'],
      dtype='object')

In [57]:
feat_imp=pd.DataFrame(data=dt0.feature_importances_.round(3),index=X_bank_sel.columns, columns=['importance'])
feat_imp.sort_values('importance', ascending=False)

,importance
Continuous interest rate (after tax),0.567
Interest Expense Ratio,0.124
Equity to Liability,0.111
Net Value Growth Rate,0.049
Operating Profit Rate,0.043
Cash Turnover Rate,0.027
Inventory/Current Liability,0.027
Realized Sales Gross Margin,0.027
Net Worth Turnover Rate (times),0.024
Borrowing dependency,0.000


# Random Forest

In [60]:
bank_rf=RandomForestClassifier(n_estimators=200, random_state=0)
bank_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [61]:
print('bank_rf on train: {:.2%}'.format(bank_rf.score(X_train, y_train)))
print('bank_rf on test: {:.2%}'.format(bank_rf.score(X_test, y_test)))

bank_rf on train: 100.00%
bank_rf on test: 96.25%


## Feature Importance

In [62]:
bank_rf.feature_importances_

array([0.03503557, 0.0426545 , 0.13722152, 0.01963926, 0.03794655,
       0.08491031, 0.0812242 , 0.12002238, 0.02225733, 0.0336561 ,
       0.03519318, 0.02242399, 0.13672654, 0.02685867, 0.01945017,
       0.02654615, 0.11823357])

In [65]:
bank_rf_feat_imp=pd.DataFrame(data=bank_rf.feature_importances_, index=X_bank_sel.columns, columns=['importance'])
bank_rf_feat_imp.sort_values(by=['importance'], ascending=False).round(3)

,importance
Continuous interest rate (after tax),0.137
Retained Earnings to Total Assets,0.137
Borrowing dependency,0.120
Equity to Liability,0.118
Net Value Growth Rate,0.085
Interest Expense Ratio,0.081
Operating Profit Rate,0.043
Cash flow rate,0.038
Inventory/Working Capital,0.035
Realized Sales Gross Margin,0.035


In [86]:
X_test.iloc[20:25,:]

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
1176,0.595007,0.998909,0.781286,8.270000e+09,0.461918,0.000404,0.628257,0.384412,0.023548,0.086678,0.278969,0.015562,0.923034,2.430000e+09,0.707643,0.013448,0.020525
948,0.610595,0.998977,0.781531,2.649232e-04,0.461711,0.000441,0.624179,0.379319,0.037258,0.004766,0.277887,0.006438,0.932147,4.840000e+09,0.909853,0.001133,0.020014
4446,0.611936,0.999137,0.781773,1.265063e-04,0.457142,0.000814,0.630616,0.378273,0.105323,0.005009,0.274562,0.002208,0.951260,3.260000e+09,0.988181,0.000884,0.016198
1756,0.623013,0.998633,0.781034,8.084147e-04,0.486569,0.000390,0.630274,0.373629,0.019355,0.000385,0.277069,0.003182,0.892959,9.330000e+07,0.486479,0.000440,0.031375
1756,0.623013,0.998633,0.781034,8.084147e-04,0.486569,0.000390,0.630274,0.373629,0.019355,0.000385,0.277069,0.003182,0.892959,9.330000e+07,0.486479,0.000440,0.031375


In [87]:
bank_rf.predict(X_test.iloc[20:25,:])

array([1, 1, 0, 1, 1])

# KNN Classifier

In [69]:
train_scores_KNN, test_scores_KNN=validation_curve(KNeighborsClassifier(), X_bank_trns, y_bank, param_name='n_neighbors', 
                                           param_range=[1,2,3,4,5,6], cv=4)

In [70]:
train_scores_KNN.mean(axis=1)

array([1.        , 0.99      , 0.93683333, 0.91983333, 0.88983333,
       0.874     ])

In [71]:
train_scores_KNN.mean()

0.9350833333333334

In [72]:
test_scores_KNN.mean()

0.86775

In [73]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel,y_bank,
                                                  random_state=0)

In [74]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=5)

In [75]:
knn_cls_bank_bal.fit(X_train,y_train)

KNeighborsClassifier()

In [76]:
print('knn_cls_bank_bal acc on train: {:.2%}'.format(knn_cls_bank_bal.score(X_train,y_train)))
print('knn_cls_bank_bal acc on test: {:.2%}'.format(knn_cls_bank_bal.score(X_test,y_test)),'\n')

knn_cls_bank_bal acc on train: 83.40%
knn_cls_bank_bal acc on test: 78.60% 



In [77]:
knn_cls_bank_bal.score(X_bank_sel,y_bank)

0.822

In [78]:
w1=[1,1,0.5,0.4,0.3,0.3,0.6,0.7,1,0.4,0.2,0.5,0.7,0.4,0.1,0.9,0.6]
knn_cls_bank_bal.predict([w1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1])

In [79]:
knn_cls_bank_bal.predict(X_bank_sel)

array([1, 0, 0, ..., 1, 1, 1])

In [80]:
len(knn_cls_bank_bal.predict(X_bank_sel))

2000

In [91]:
predict_array = knn_cls_bank_bal.predict(X_bank_sel)
count_non_zero = 0
for i in predict_array:
    if i != 0:
        count_non_zero += 1
print(count_non_zero)      

1198


In [95]:
sum(knn_cls_bank_bal.predict(X_bank_sel))

1198

In [85]:
X_bank_sel.describe()

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000
mean,0.603785,0.998843,0.781041,1.844045e+09,0.464422,3.732000e+07,0.631498,0.382959,0.039831,3.480000e+06,0.277368,3.585150e+07,0.919430,2.332955e+09,0.748211,7.206000e+07,0.034478
std,0.014686,0.000993,0.003344,3.190004e+09,0.012146,5.890469e+08,0.014577,0.052685,0.036178,1.100197e+08,0.005311,4.840476e+08,0.034846,2.687308e+09,0.204472,7.652163e+08,0.048426
min,0.449761,0.973424,0.735093,1.000256e-04,0.343818,0.000000e+00,0.525126,0.000000,0.009032,0.000000e+00,0.244753,0.000000e+00,0.728885,0.000000e+00,0.049582,1.015558e-04,0.003946
25%,0.597580,0.998857,0.781286,1.549529e-04,0.459606,3.896315e-04,0.630095,0.371918,0.021935,4.761171e-03,0.276920,2.487101e-03,0.909662,7.248945e-04,0.617815,9.988970e-04,0.017979
50%,0.602099,0.998976,0.781541,2.864894e-04,0.462599,4.406694e-04,0.630612,0.376805,0.029839,1.027135e-02,0.277135,5.344259e-03,0.930045,1.270000e+09,0.792352,2.623103e-03,0.023615
75%,0.609738,0.999044,0.781652,3.202500e+09,0.466664,4.735780e-04,0.631087,0.383465,0.043750,2.309766e-02,0.277511,9.879049e-03,0.938763,3.952500e+09,0.919076,6.552762e-03,0.035823
max,0.665151,0.999650,0.786079,9.980000e+09,0.617039,9.330000e+09,0.892562,1.000000,0.404839,3.480000e+09,0.466868,9.910000e+09,0.998858,1.000000e+10,1.000000,9.650000e+09,0.942729
